In [1]:
import os
from vllm.steer_vectors.request import SteerVectorRequest
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/",
    enable_steer_vector=True,
    enforce_eager=True,
    tensor_parallel_size=1,
    enable_chunked_prefill=False
)

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/"
)

INFO 11-04 00:33:18 [utils.py:253] non-default args: {'disable_log_stats': True, 'enforce_eager': True, 'enable_steer_vector': True, 'enable_chunked_prefill': False, 'model': '/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/'}
INFO 11-04 00:33:18 [model.py:657] Resolved architecture: Qwen2ForCausalLM
INFO 11-04 00:33:18 [model.py:1746] Using max model len 32768
INFO 11-04 00:33:20 [scheduler.py:211] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 11-04 00:33:20 [vllm.py:414] Cudagraph is disabled under eager mode
(EngineCore_DP0 pid=2691009) INFO 11-04 00:33:21 [core.py:94] Initializing a V1 LLM engine (v0.1.dev10888+g9d4fd0da4.d20251031) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768,

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=2691009) INFO 11-04 00:33:25 [default_loader.py:314] Loading weights took 0.84 seconds
(EngineCore_DP0 pid=2691009) INFO 11-04 00:33:25 [steer_vector_model_runner_mixin.py:36] Initialized SteerVector worker manager
(EngineCore_DP0 pid=2691009) INFO 11-04 00:33:25 [steer_vector_model_runner_mixin.py:50] Wrapping model with steer vector support
(EngineCore_DP0 pid=2691009) INFO 11-04 00:33:25 [hidden_states_model_runner_mixin.py:90] Wrapped 28 decoder layers for hidden states capture
(EngineCore_DP0 pid=2691009) INFO 11-04 00:33:26 [gpu_model_runner.py:2971] Model loading took 2.8876 GiB and 0.996471 seconds
(EngineCore_DP0 pid=2691009) INFO 11-04 00:33:27 [gpu_worker.py:343] Available KV cache memory: 38.36 GiB
(EngineCore_DP0 pid=2691009) INFO 11-04 00:33:28 [kv_cache_utils.py:1247] GPU KV cache size: 1,436,560 tokens
(EngineCore_DP0 pid=2691009) INFO 11-04 00:33:28 [kv_cache_utils.py:1252] Maximum concurrency for 32,768 tokens per request: 43.84x
(EngineCore_DP0 pi

In [2]:
prompt_template = "<|im_start|>user\n%s<|im_end|>\n<|im_start|>assistant\n"
prompt = "Find the greatest common divisor of $3339$, $2961$, and $1491$."
text = prompt_template % prompt

output = llm.generate(
    text,
    SamplingParams(
        temperature=0,
        max_tokens=4096,
        skip_special_tokens=False,
    ),
)
print(output[0].outputs[0].text)
print("Baseline tokens: ", len(tokenizer.tokenize(output[0].outputs[0].text, add_special_tokens=True)))

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, 

To find the greatest common divisor (GCD) of the numbers 3339, 2961, and 1491, we will use the Euclidean algorithm. The Euclidean algorithm involves repeated division and taking remainders until we reach a remainder of 0. The last non-zero remainder is the GCD.

First, we find the GCD of 3339 and 2961.

1. Divide 3339 by 2961 and find the remainder:
   \[
   3339 = 2961 \cdot 1 + 378
   \]
   So, the remainder is 378.

2. Now, we need to find the GCD of 2961 and 378. Divide 2961 by 378:
   \[
   2961 = 378 \cdot 7 + 255
   \]
   So, the remainder is 255.

3. Next, we find the GCD of 378 and 255. Divide 378 by 255:
   \[
   378 = 255 \cdot 1 + 123
   \]
   So, the remainder is 123.

4. Now, we find the GCD of 255 and 123. Divide 255 by 123:
   \[
   255 = 123 \cdot 2 + 0
   \]
   So, the remainder is 0. The last non-zero remainder is 123.

Thus, the GCD of 3339 and 2961 is 123.

Next, we need to find the GCD of 123 and 1491.

1. Divide 1491 by 123:
   \[
   1491 = 123 \cdot 12 + 15
   \

In [3]:
steer_vector_request_pos = SteerVectorRequest(
    steer_vector_name="thinking_switch_pos",
    steer_vector_int_id=1,
    steer_vector_local_path="MATH500.gguf",
    scale=2.0,
    prefill_trigger_tokens=[-1],
    generate_trigger_tokens=[-1],
    debug=False,
    algorithm='direct',
    target_layers=list(range(28))
)
output = llm.generate(
    text,
    SamplingParams(
        temperature=0,
        max_tokens=4096,
        skip_special_tokens=False,
    ),
    steer_vector_request=steer_vector_request_pos
)
print(output[0].outputs[0].text)
print("Slow tokens: ", len(tokenizer.tokenize(output[0].outputs[0].text, add_special_tokens=True)))

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, 

To find the greatest common divisor (GCD) of the numbers 3339, 2961, and 1491, we will use the Euclidean algorithm. The Euclidean algorithm is a method for finding the GCD of two numbers by repeatedly applying the division algorithm. We will extend this method to three numbers by breaking it down into steps.

### Step 1: GCD of 3339 and 2961
First, we find the GCD of 3339 and 2961.

1. **Step 1.1: Divide 3339 by 2961 and find the remainder.**
   \[
   3339 \div 2961 \approx 1.123 \quad \text{(not an integer, so we need to adjust our approach)}
   \]
   Let's try a different approach by using the Euclidean algorithm directly.

2. **Step 1.2: Use the Euclidean algorithm for 3339 and 2961.**
   \[
   \begin{align*}
   3339 &= 2961 \times 1 + 378 \\
   2961 &= 378 \times 7 + 375 \\
   378 &= 375 \times 1 + 3 \\
   375 &= 375 \times 1 + 0
   \end{align*}
   \]
   The Euclidean algorithm shows that the GCD of 3339 and 2961 is 3.

### Step 2: GCD of 3339 and 375
Now, we need to find the GCD o

In [4]:
steer_vector_request_neg = SteerVectorRequest(
    steer_vector_name="thinking_switch_neg",
    steer_vector_int_id=2,
    steer_vector_local_path="MATH500.gguf",
    scale=-2.0,
    prefill_trigger_tokens=[-1],
    generate_trigger_tokens=[-1],
    debug=False,
    algorithm='direct',
    target_layers=list(range(28))
)
output = llm.generate(
    text,
    SamplingParams(
        temperature=0,
        max_tokens=4096,
        skip_special_tokens=False,
    ),
    steer_vector_request=steer_vector_request_neg
)
print(output[0].outputs[0].text)
print("Fast tokens: ", len(tokenizer.tokenize(output[0].outputs[0].text, add_special_tokens=True)))

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, 

To find the greatest common divisor (GCD) of \(3339\), \(2961\), and \(1491\), we start by determining the prime factorization of each number.

First, we factorize \(3339\):
\[3339 = 3 \times 1113 = 3 \times 3 \times 371 = 3 \times 3 \times 7 \times 53 = 3^2 \times 7 \times 53.\]

Next, we factorize \(2961\):
\[2961 = 3 \times 987 = 3 \times 3 \times 329 = 3^2 \times 7 \times 53.\]

Finally, we factorize \(1491\):
\[1491 = 3 \times 497 = 3 \times 7 \times 71 = 3 \times 7^2 \times 71.\]

The common factors of \(3339\), \(2961\), and \(1491\) are \(3\) and \(7\). Therefore, the GCD is:
\[3 \times 7 = 21.\]

Thus, the greatest common divisor of \(3339\), \(2961\), and \(1491\) is \(\boxed{21}\).
Fast tokens:  316
